In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

from IPython.display import display
import json
from pathlib import Path
from pprint import pprint
from typing import Optional

from _secrets import CLAUDE_API_KEY, ELEVENLABS_API_KEY, OPENAI_API_KEY
from gdutils.audios import play_mp3
from gdutils.dicts import print_dict
from gdutils.llms_claude import call_claude_gpt
from gdutils.outloud_text_to_speech import outloud_elevenlabs
from vocab_llm_utils import (
    add_delays,
    extract_text_from_image,
    metadata_for_word_full,
    metadata_for_word_full_cached,
    generate_txt_audio,
    glob_images,
    load_logd,
    play_txt_audio,
    process_img_filens,
    pull_out_tricky_words_or_phrases,
    quick_search_for_word,
    # read_word_json,
    save_logd,
    # write_word_json,

)

DIRN = "/Users/greg/Library/Mobile Documents/3L68KQB4HG~com~readdle~CommonDocuments/Documents/Greek learning/Greek learning - Alexandra/241118/"
# WRITE_DIRN = "/Users/greg/Dropbox/dev/spideryarn/241103_vocab/HelloZenno/HelloZenno/data"

In [18]:
logd, logd_filen = load_logd(DIRN)

In [ ]:
img_filens = glob_images(DIRN)

# txt_el, extra = extract_greek_text_from_image(img_filens=img_filens, verbose=2)
# generate_txt_audio(txt_el, DIRN)
# print(txt_el)

for img_filen in img_filens:
    # if img_filen in logd["sources"]:
    #     print(f"Already processed {img_filen}")
    #     continue
    
    already_words = [d["original_word"] for d in logd["words"]]
    print(f"Ignoring: {already_words}")
    source_d, this_words, extra = process_img_filens(
        img_filen, DIRN, ignore_words=already_words, verbose=1
    )

    print(f"New words: {[d['original_word'] for d in this_words]}")

    logd["sources"][img_filen] = source_d
    logd["words"].extend(this_words)

# extra_processed = process_img_filens(img_filens, DIRN, verbose=1)

In [15]:
save_logd(DIRN, logd)

In [ ]:
word_d, extra = metadata_for_word_full("αρσενικός ", verbose=1)
word_d, extra = quick_search_for_greek_word("αρσενικός", verbose=1)

In [ ]:
tricky, extra = pull_out_tricky_greek_words_or_phrases(txt_el, ignore_words=None, verbose=1)
len(tricky['words'])

In [ ]:
from vocab_utils import get_greek_wiktionary_frequency_list, get_greek_wiktionary_frequency_list_cached

out = get_greek_wiktionary_frequency_list_cached()
pprint(out[:10])

In [ ]:
txt = "This is a test, this is a test. This is a\ntest"
# txt_slow = add_delays(txt)
# print(txt_slow)

mp3_filen = generate_txt_audio(txt, '.', should_add_delays=True, should_play=True)
print(mp3_filen)

In [ ]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scopes
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1ALbp_UXBR9Nmc0_8dM4HqNwSRGg05gzEEZtqL6_VmYc/edit?gid=0#gid=0"
GOOGLE_SHEET_ID = GOOGLE_SHEET_URL.split("/")[-2]

GOOGLE_OAUTH_FLOW_PORT = 8080

from config import GOOGLE_CLIENT_CREDENTIALS

GOOGLE_CLIENT_TOKEN_FILEN = "credentials/google_client_token.json"


def write_to_google_sheet(sheet_id: str):
    """Shows basic usage of the Sheets API."""
    creds = None
    # The token.json file stores the user's access and refresh tokens.
    if os.path.exists(GOOGLE_CLIENT_TOKEN_FILEN):
        creds = Credentials.from_authorized_user_file(GOOGLE_CLIENT_TOKEN_FILEN, SCOPES)
    # If credentials are not available or are invalid, log the user in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                GOOGLE_CLIENT_CREDENTIALS,
                SCOPES,
                # redirect_uri="http://localhost:3000/oauth2callback",
            )
            flow.redirect_uri = 'http://localhost:8080/oauth2callback'
            print("Redirect URI being used:", flow.redirect_uri)
            creds = flow.run_local_server(
                # port=GOOGLE_OAUTH_FLOW_PORT,
                # host="localhost",
                # redirect_uri_trailing_slash=False,
                # redirect_uri_path="oauth2callback",
            )
        # Save the credentials for future use
        with open(GOOGLE_CLIENT_TOKEN_FILEN, "w") as token:
            token.write(creds.to_json())

    # Initialize the Sheets API service
    service = build("sheets", "v4", credentials=creds)

    # The range of cells to update
    range_name = "Sheet1!A1:B2"

    # The new values to write
    values = [["Hello", "World"], ["Python", "Rocks"]]
    body = {"values": values}

    # Update the sheet
    result = (
        service.spreadsheets()
        .values()
        .update(
            spreadsheetId=sheet_id, range=range_name, valueInputOption="RAW", body=body
        )
        .execute()
    )
    print(f"{result.get('updatedCells')} cells updated.")


write_to_google_sheet(GOOGLE_SHEET_ID)

In [87]:

GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1ALbp_UXBR9Nmc0_8dM4HqNwSRGg05gzEEZtqL6_VmYc/edit?gid=0#gid=0"
GOOGLE_SHEET_ID = GOOGLE_SHEET_URL.split("/")[-2]
GOOGLE_CLIENT_TOKEN_FILEN = "credentials/google_client_token.json"


def write_to_google_sheet(sheet_id: str):
    creds = None
    creds = Credentials.from_authorized_user_file(GOOGLE_CLIENT_TOKEN_FILEN, SCOPES)
    # If credentials are not available or are invalid, log the user in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
            token.write(creds.to_json())
        else:
            raise Exception("Credentials are not valid")

    # Initialize the Sheets API service
    service = build("sheets", "v4", credentials=creds)

    # The range of cells to update
    range_name = "Sheet1!A1:B2"

    # The new values to write
    values = [["Hello", "World"], ["Python", "Rocks"]]
    body = {"values": values}

    # Update the sheet
    result = (
        service.spreadsheets()
        .values()
        .update(
            spreadsheetId=sheet_id, range=range_name, valueInputOption="RAW", body=body
        )
        .execute()
    )
    print(f"{result.get('updatedCells')} cells updated.")


write_to_google_sheet(GOOGLE_SHEET_ID)

FileNotFoundError: [Errno 2] No such file or directory: 'credentials/google_client_token.json'